## Import các thư viện cần thiết

In [1]:
import datetime as dt # Để làm việc với dữ liệu datetime
import math

## Đưa ra câu hỏi cần trả lời

Tình trạng hiểu về Python các bạn trong lớp?

## Thu thập dữ liệu

Cho làm quiz về Python trên moodle.

Kết quả có thể được download về dưới dạng file .csv.

## Khám phá dữ liệu

In [4]:
file = open('Data/PythonQuiz.csv', 'r')

cols = {}
first_line_vals = file.readline().rstrip().split(',')
for first_line_val in first_line_vals:
    cols[first_line_val] = []
for line in file:
    line_vals = line.rstrip().split(',')
    for i in range(len(line_vals)):
        cols[first_line_vals[i]].append(line_vals[i])

file.close()

cols

FileNotFoundError: [Errno 2] No such file or directory: 'Data/PythonQuiz_2024_CQ.csv'

### Dữ liệu có bao nhiêu dòng và bao nhiêu cột?

In [ ]:
cols.keys()

In [ ]:
# Số dòng
len(cols['State'])

In [ ]:
# Số cột
len(cols)

### Mỗi dòng có ý nghĩa gì? Có vấn đề các dòng có ý nghĩa khác nhau không?

Mỗi dòng ứng với kết quả làm quiz của một sinh viên.

Xem qua bảng dữ liệu của file csv thì có vẻ không thấy có dòng nào "lạc loài" (thật ra dữ liệu ban đầu là có dòng cuối là dòng tính điểm trung bình của các cột, nhưng mình đã bỏ đi rồi).

### Dữ liệu có các dòng bị lặp không?

Xem qua bảng dữ liệu của file csv thì có vẻ không thấy có vấn đề một sinh viên xuất hiện nhiều hơn một lần.

### Mỗi cột có ý nghĩa gì?

- Cột "State": cho biết trạng thái làm bài, chẳng hạn "Finished" là đã nộp bài (đã ấn nút "submit all and finish")
- 3 cột "Started on", "Completed", "Time taken": lần lượt cho biết thời điểm bắt đầu làm, thời điểm làm xong, và thời gian từ khi bắt đầu làm cho tới khi làm xong
- Cột "Grade/16.00": cho biết điểm tổng (tối đa là 16, vì có 16 câu và mỗi câu 1 điểm)
- 16 cột còn lại "Q. 1 /1.00", "Q. 2 /1.00", ...: cho biết điểm của mỗi câu quiz (mỗi câu làm đúng hoàn toàn thì sẽ được 1 điểm)

### Mỗi cột hiện đang có kiểu dữ liệu gì? Có cột nào có kiểu dữ liệu chưa phù hợp để có thể xử lý tiếp không?

In [ ]:
for col_name, col_vals in cols.items():
    print(f'{col_name} {type(col_vals[0])}')

Đúng như dự đoán, tất cả các cột đều đang có kiểu dữ liệu là str. Để có thể tiếp tục khám phá tiếp thì ta cần tiền xử lý để các cột có kiểu dữ liệu phù hợp.

- Cột "State": string là đúng rồi, không cần chuyển
- Cột "Started on" và "Completed on": chuyển sang kiểu dữ liệu datetime
- Cột "Time taken": chuyển sang một con số thể hiện cho số phút
- Các cột "Grade/16.00", "Q. 1 /1.00", ...: chuyển sang kiểu dữ liệu số

## Tiền xử lý

In [ ]:
new_cols = {}
for col_name, col_vals in cols.items():
    if '/' in col_name:
        new_col_vals = []
        for col_val in col_vals:
            new_col_vals.append(float(col_val))
        new_cols[col_name] = new_col_vals
    elif col_name in ['Started on', 'Completed']:
        new_col_vals = []
        for col_val in col_vals:
            new_col_val = dt.datetime.strptime(col_val,
                                               '%d %B %Y %H:%M %p')
            new_col_vals.append(new_col_val)
        new_cols[col_name] = new_col_vals
    elif col_name == 'Time taken':
        new_col_vals = []
        for col_val in col_vals:
            hour = 0; minute = 0; second = 0
            temp_list = col_val.split(' ')
            for temp_i in range(len(temp_list)):
                if 'hour' in temp_list[temp_i]:
                    hour = int(temp_list[temp_i - 1])
                elif 'min' in temp_list[temp_i]:
                    minute = int(temp_list[temp_i - 1])
                elif 'sec' in temp_list[temp_i]:
                    second = int(temp_list[temp_i - 1])
            new_col_val = hour * 60 + minute + second / 60
            new_col_vals.append(new_col_val)
        new_cols[col_name] = new_col_vals
    else:
        new_cols[col_name] = col_vals

In [ ]:
cols = new_cols

In [ ]:
for col_name, col_vals in cols.items():
    print(f'{col_name} {type(col_vals[0])}')

## Khám phá dữ liệu

### Với mỗi cột có kiểu dữ liệu dạng số (numerical), các giá trị được phân bố như thế nào?

- Số-lượng/tỉ-lệ các giá trị thiếu?
- Min? max? $\to$ Có gì bất thường không?

Ta thấy: cột "Started on" và "Completed" có kiểu dữ liệu datetime nhưng cũng có thể cho gia nhập hội numeric và tính min & max luôn.

Số-lượng/tỉ-lệ các giá trị thiếu? Không có giá trị thiếu.

Min? Max?

In [ ]:
for col_name, col_vals in cols.items():
    if type(col_vals[0]) != str:
        print(f'{col_name}\t{min(col_vals)}\t{max(col_vals)}')

Có vẻ không có gì bất thường.

### Với mỗi cột có kiểu dữ liệu dạng phân loại (categorical), các giá trị được phân bố như thế nào?

- Số-lượng/tỉ-lệ các giá trị thiếu?
- Số lượng các giá trị khác nhau? Show một vài giá trị \
$\to$ Có gì bất thường không?

Hội này chỉ có một thành viên là cột "State"!

Số-lượng/tỉ-lệ các giá trị thiếu? Không có giá trị thiếu.

Số lượng các giá trị khác nhau? Show một vài giá trị

In [ ]:
for col_name, col_vals in cols.items():
    if type(col_vals[0]) == str:
        col_vals_set = set(col_vals)
        print(f'{col_name}\t{len(col_vals_set)}\t{col_vals_set}')

Có vẻ không có gì bất thường.

## Đưa ra câu hỏi cần trả lời

Sau khi đã hiểu hơn về dữ liệu, ta sẽ quay lại bước đưa ra câu hỏi cần trả lời: tinh chỉnh câu hỏi, hoặc có thể đưa ra thêm các câu hỏi mới.

Câu hỏi “tình trạng hiểu về Python của các bạn trong lớp?” có thể được cụ thể hóa ra thành 2 câu hỏi:

1. Cột điểm tổng “Grade/16.00” có phân bố các giá trị như thế nào?
2. Câu quiz nào có nhiều bạn làm sai nhất, câu nào có nhiều bạn làm sai nhì, ...?

## Tiền xử lý + phân tích dữ liệu để trả lời cho câu hỏi 1

*Câu hỏi 1: Cột điểm tổng “Grade/16.00” có phân bố các giá trị như thế nào?*

Ở đây không cần tiền xử lý mà sẽ đi luôn vào phân tích dữ liệu.

Đầu tiên, ta sẽ xem thử median của cột "Grade/16.00".

In [ ]:
grades = cols['Grade/16.00']
sorted_grades = sorted(grades)
p = 50
i = int(p/100 * (len(sorted_grades)-1))
sorted_grades[i]

Để cảm thụ rõ hơn về phân bố các giá trị của cột "Grade/16.00", ta sẽ chia miền giá trị của cột này thành các khoảng (bin) và xem thử mỗi khoảng có bao nhiêu sinh viên.

Giả sử ta sẽ chia đoạn [0, 16] thành 4 bin như sau: [0, 4], (4, 8], (8, 12], (12, 16]. Ta có thể lưu kết quả vào một dictionary, trong đó: key là một tuple cho biết 2 đầu của một bin (thay tuple bằng list thì có được không?), value là số lượng sinh viên rơi vào bin này.

***Sinh viên hoàn thành tiếp phần này như bài tập về nhà***

## Tiền xử lý + phân tích dữ liệu để trả lời cho câu hỏi 2

*Câu hỏi 2: Câu quiz nào có nhiều bạn làm sai nhất, câu nào có nhiều bạn làm sai nhì, ...?*

Ta có thể lưu kết quả vào một dictionary, trong đó: key là tên của cột ứng với câu quiz, value là điểm trung bình của cột. Dictionary này được sort tăng dần theo value.

***Sinh viên hoàn thành tiếp phần này như bài tập về nhà***